# Producción y distribución — Modelo y Endpoint

Optimización lineal (LP) para coordinar producción de papas fritas y asignación a Centros de Distribución (CD), y expone un endpoint **FastAPI** para consultar planes dado un set de parámetros (demanda, capacidades y costos).

In [19]:
# pip install pulp fastapi uvicorn pydantic pandas

In [4]:
import json, math
from typing import Dict, List
import pandas as pd
import pulp

In [2]:
plants = pd.DataFrame({
    "plant": ["PlantA", "PlantB"],
    "capacity_kg": [120_000, 90_000],
    "prod_cost_per_kg": [1.10, 1.25],
    "yield": [0.95, 0.93],  # rendimiento neto (después de mermas)
})

dcs = pd.DataFrame({
    "dc": ["CD_Norte", "CD_Centro", "CD_Sur"],
    "demand_kg": [100_000, 60_000, 30_000]
})

# matriz de costos de transporte ($/kg)
transport = pd.DataFrame([
    ["PlantA", "CD_Norte", 0.20],
    ["PlantA", "CD_Centro", 0.25],
    ["PlantA", "CD_Sur",   0.35],
    ["PlantB", "CD_Norte", 0.30],
    ["PlantB", "CD_Centro",0.22],
    ["PlantB", "CD_Sur",   0.28],
], columns=["plant","dc","transp_cost_per_kg"])

penalty_stockout = 3.00  # $/kg no servido (proxy de penalización/SLA)

In [3]:
plants, dcs, transport.head()

(    plant  capacity_kg  prod_cost_per_kg  yield
 0  PlantA       120000              1.10   0.95
 1  PlantB        90000              1.25   0.93,
           dc  demand_kg
 0   CD_Norte     100000
 1  CD_Centro      60000
 2     CD_Sur      30000,
     plant         dc  transp_cost_per_kg
 0  PlantA   CD_Norte                0.20
 1  PlantA  CD_Centro                0.25
 2  PlantA     CD_Sur                0.35
 3  PlantB   CD_Norte                0.30
 4  PlantB  CD_Centro                0.22)

## Modelo LP (PuLP)

**Decisiones**  
- `produce[p]` = kg a producir en planta *p*  
- `ship[p,d]` = kg enviados de planta *p* al CD *d*

**Objetivo**  
Minimizar `costo = producción + transporte + penalización por faltantes`

**Restricciones**  
- Capacidad por planta: `produce[p] ≤ capacity[p] * yield[p]`  
- Balance planta: `produce[p] ≥ ∑_d ship[p,d]`  
- Servicio por CD: `∑_p ship[p,d] + short[d] = demanda[d]` con `short[d] ≥ 0`

In [5]:
# Conjuntos
P = plants["plant"].tolist()
D = dcs["dc"].tolist()

cap = dict(zip(plants["plant"], plants["capacity_kg"]*plants["yield"]))
c_prod = dict(zip(plants["plant"], plants["prod_cost_per_kg"]))

c_transp = {(r.plant, r.dc): r.transp_cost_per_kg for r in transport.itertuples(index=False)}
demand = dict(zip(dcs["dc"], dcs["demand_kg"]))

In [6]:
# Modelo
m = pulp.LpProblem("Snack_Prod_Distrib", pulp.LpMinimize)

In [7]:
# Variables
produce = pulp.LpVariable.dicts("produce", P, lowBound=0)
ship = pulp.LpVariable.dicts("ship", [(p,d) for p in P for d in D], lowBound=0)
short = pulp.LpVariable.dicts("short", D, lowBound=0)

In [8]:
# Objetivo
m += (
    pulp.lpSum(c_prod[p]*produce[p] for p in P) +
    pulp.lpSum(c_transp.get((p,d), 1e6)*ship[(p,d)] for p in P for d in D) +
    penalty_stockout*pulp.lpSum(short[d] for d in D)
)

In [9]:
# Capacidad
for p in P:
    m += produce[p] <= cap[p], f"cap_{p}"

In [10]:
# Balance planta
for p in P:
    m += produce[p] >= pulp.lpSum(ship[(p,d)] for d in D), f"balance_{p}"

In [11]:
# Satisfacción de demanda
for d in D:
    m += pulp.lpSum(ship[(p,d)] for p in P) + short[d] == demand[d], f"demand_{d}"

In [12]:
# Resolver
m.solve(pulp.PULP_CBC_CMD(msg=False))

status = pulp.LpStatus[m.status]
total_cost = pulp.value(m.objective)

print("Status:", status)
print("Costo total:", round(total_cost,2))

Status: Optimal
Costo total: 262420.0


In [13]:
# Extraer resultados
df_produce = pd.DataFrame({
    "plant": P,
    "produce_kg": [produce[p].value() for p in P]
})

rows = []
for p in P:
    for d in D:
        val = ship[(p,d)].value()
        if val and val > 1e-6:
            rows.append([p,d,val])
df_ship = pd.DataFrame(rows, columns=["plant","dc","ship_kg"])

df_short = pd.DataFrame({
    "dc": D,
    "short_kg": [short[d].value() for d in D]
})

df_produce, df_ship, df_short

(    plant  produce_kg
 0  PlantA    114000.0
 1  PlantB     76000.0,
     plant         dc   ship_kg
 0  PlantA   CD_Norte  100000.0
 1  PlantA  CD_Centro   14000.0
 2  PlantB  CD_Centro   46000.0
 3  PlantB     CD_Sur   30000.0,
           dc  short_kg
 0   CD_Norte       0.0
 1  CD_Centro       0.0
 2     CD_Sur       0.0)

## Persistencia de parámetros

Guardamos parámetros base (capacidades, costos, etc.) para que el endpoint pueda reutilizarlos por defecto.

In [14]:
config = {
    "plants": plants.to_dict(orient="records"),
    "dcs": dcs.to_dict(orient="records"),
    "transport": transport.to_dict(orient="records"),
    "penalty_stockout": penalty_stockout
}

In [15]:
with open("model_config.json","w") as f:
    json.dump(config, f, indent=2)
print("Guardado: model_config.json")

Guardado: model_config.json


## Endpoint FastAPI

- **POST** `/optimize`  
  Cuerpo JSON con (opcional) nuevos parámetros; si faltan, se usan los valores por defecto en `model_config.json`.

In [16]:
# Creamos un archivo app.py con el servicio FastAPI
app_code = r'''
import json
from typing import List, Dict, Optional
from fastapi import FastAPI
from pydantic import BaseModel
import pandas as pd
import pulp

# Cargar config por defecto
with open("model_config.json","r") as f:
    DEFAULT = json.load(f)

class Plant(BaseModel):
    plant: str
    capacity_kg: float
    prod_cost_per_kg: float
    yield_: float = 1.0

class DC(BaseModel):
    dc: str
    demand_kg: float

class Transport(BaseModel):
    plant: str
    dc: str
    transp_cost_per_kg: float

class OptimizeRequest(BaseModel):
    plants: Optional[List[Plant]] = None
    dcs: Optional[List[DC]] = None
    transport: Optional[List[Transport]] = None
    penalty_stockout: Optional[float] = None

app = FastAPI(title="Snack Production & Distribution Optimizer")

def to_df(lst, cols):
    if lst is None: return None
    df = pd.DataFrame([x.dict() for x in lst])
    # Renombrar yield_->yield para consistencia interna
    if "yield_" in df.columns:
        df = df.rename(columns={"yield_":"yield"})
    return df[cols]

@app.post("/optimize")
def optimize(req: OptimizeRequest):
    plants = to_df(req.plants, ["plant","capacity_kg","prod_cost_per_kg","yield"]) \
             if req.plants is not None else pd.DataFrame(DEFAULT["plants"])
    dcs = to_df(req.dcs, ["dc","demand_kg"]) \
          if req.dcs is not None else pd.DataFrame(DEFAULT["dcs"])
    transport = to_df(req.transport, ["plant","dc","transp_cost_per_kg"]) \
                if req.transport is not None else pd.DataFrame(DEFAULT["transport"])
    penalty = req.penalty_stockout if req.penalty_stockout is not None else DEFAULT["penalty_stockout"]

    P = plants["plant"].tolist()
    D = dcs["dc"].tolist()

    cap = dict(zip(plants["plant"], plants["capacity_kg"]*plants["yield"]))
    c_prod = dict(zip(plants["plant"], plants["prod_cost_per_kg"]))
    c_transp = {(r.plant, r.dc): r.transp_cost_per_kg for r in transport.itertuples(index=False)}
    demand = dict(zip(dcs["dc"], dcs["demand_kg"]))

    m = pulp.LpProblem("Snack_Prod_Distrib", pulp.LpMinimize)

    produce = pulp.LpVariable.dicts("produce", P, lowBound=0)
    ship = pulp.LpVariable.dicts("ship", [(p,d) for p in P for d in D], lowBound=0)
    short = pulp.LpVariable.dicts("short", D, lowBound=0)

    m += (
        pulp.lpSum(c_prod[p]*produce[p] for p in P) +
        pulp.lpSum(c_transp.get((p,d), 1e6)*ship[(p,d)] for p in P for d in D) +
        penalty*pulp.lpSum(short[d] for d in D)
    )

    for p in P:
        m += produce[p] <= cap[p]
        m += produce[p] >= pulp.lpSum(ship[(p,d)] for d in D)

    for d in D:
        m += pulp.lpSum(ship[(p,d)] for p in P) + short[d] == demand[d]

    m.solve(pulp.PULP_CBC_CMD(msg=False))

    res = {
        "status": pulp.LpStatus[m.status],
        "total_cost": pulp.value(m.objective),
        "produce": [{ "plant": p, "produce_kg": produce[p].value() } for p in P],
        "ship": [{ "plant": p, "dc": d, "ship_kg": ship[(p,d)].value() } for p in P for d in D],
        "short": [{ "dc": d, "short_kg": short[d].value() } for d in D]
    }
    # Filtrar pequeñas tolerancias numéricas
    res["ship"] = [r for r in res["ship"] if r["ship_kg"] and r["ship_kg"] > 1e-6]
    return res
'''

In [17]:
with open("app.py","w", encoding="utf-8") as f:
    f.write(app_code)

In [18]:
print("Archivo 'app.py' generado. Para correr el servidor local:")
print("uvicorn app:app --reload --port 8000")

Archivo 'app.py' generado. Para correr el servidor local:
uvicorn app:app --reload --port 8000
